<a href="https://colab.research.google.com/github/kangwonlee/pytorch-ibm-coursera/blob/main/week06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello PyTorch 👋🏻



references
* https://www.coursera.org/learn/deep-neural-networks-with-pytorch/
* https://github.com/damounayman/Deep-Neural-Networks-with-PyTorch/blob/main/Week1/1D_tensors.ipynb



## week 6



### 9.1 Convolution



### 9.2 Activation Functions and Max Polling



### 9.3 Multiple Input and Output Channels



### 9.4 Convolutional Neural Network



In [ ]:
import os
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn
import torch.optim
import torch.utils.data



In [ ]:
torch.manual_seed(4)



CNN constructor class



In [ ]:
class CNN(torch.nn.Module):
  def __init__(self, out_1:int=2, out_2:int=1):
    super(CNN, self).__init__()
    self.cnn1 = torch.nn.Conv2d(
      in_channels=1,      # grayscale
      out_channels=out_1,
      kernel_size=2, padding=0
    )

    self.maxpool1 = torch.nn.MaxPool2d(kernel_size=2, stride=1)

    self.cnn2 = torch.nn.Conv2d(
      in_channels=out_1,
      out_channels=out_2,
      kernel_size=2, padding=0
    )

    self.maxpool2 = torch.nn.MaxPool2d(kernel_size=2, stride=1)

    # TODO : check M - K + 1
    self.fc1 = torch.nn.Linear(
      out_2*7*7,
      2, # one for each class
    )

  def forward(self, x:torch.tensor):
    x = self.cnn1(x)
    # TODO : is it common to use maxpooling after activation function?
    x = torch.relu(x)
    x = self.maxpool1(x)

    x = self.cnn2(x)
    x = torch.relu(x)
    x = self.maxpool2(x)

    x = x.view(x.size(0), -1)
    x = self.fc1(x)

    return x



In [ ]:
class Data(torch.utils.data.Dataset):
  def __init__(self, N_images:int=100, offset=0, p:float=0.9, train:bool=False):
    '''
    p: probability of white pixel
    N_images : # images
    offset : int
    '''
    if train:
      np.random.seed(1)

    N_images = 2 * (N_images // 2)
    assert 0 == (N_images % 2), f'N_images = {N_images}'

    images = np.zeros((N_images, 1, 11, 11))

    start1 = 3
    start2 = 1

    self.y = torch.zeros(N_images).type(torch.long)

    for n in range(N_images):
      if 0 < offset:
        low = int(
            np.random.randint(
                low=start1,
                high=start1+offset,
                size=1
            )
        )
        high = int(
            np.random.randint(
                low=start2,
                high=start2+offset,
                size=1
            )
        )
      else:
        low = 4
        high = 1
      # end if offset block

      if n <= (N_images // 2):
        self.y[n] = 0
        images[
            n, 0, high:high+9, low:low+3
        ] = np.random.binomial(1, p, (9, 3))
      elif n > (N_images // 2):
        self.y[n] = 1
        images[
            n, 0, low:low+3, high:high+9
        ] = np.random.binomial(1, p, (3, 9))
      # end if n block
    # end for loop

    self.x = torch.from_numpy(images).type(torch.FloatTensor)
    self.len = self.x.shape[0]
    del images
    np.random.seed(0)
  # end __init__()

  def __getitem__(self, index:int):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.len



In [ ]:
N_images = 10000
train_dataset = Data(N_images=N_images, train=True)
validation_dataset = Data(N_images=N_images, train=False)

model = CNN(2, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

criterion = torch.nn.CrossEntropyLoss()
train_loader = torch.utils.data.DataLoader(
  dataset=train_dataset,
  batch_size=100,
)

validation_loader = torch.utils.data.DataLoader(
  dataset=validation_dataset,
  batch_size=5000,
)



In [ ]:
def train_validate(
    model, optimizer, criterion,
    train_loader,
    validation_loader=[],
    n_epoch:int=1000,
  ) -> Dict[str, List[float]]:
  record = {
    'training_loss': [],
    'validation_accuracy': [],
  }
  if os.getenv('CI', False):
    n_epoch = 1

  for epoch in range(n_epoch):
    total = 0.0

    for x, y in train_loader:
      optimizer.zero_grad()

      yhat = model(x)

      loss = criterion(yhat, y)
      optimizer.zero_grad()
      loss.backward()

      optimizer.step()

      total += loss.item()

      record['training_loss'].append(loss.item())

    # end train_loader loop

    correct = 0
    population = 0
    for x, y in validation_loader:
      z = model(x)
      _, label = torch.max(z, 1)
      correct += (label==y).sum().item()
      population += len(y)
    # end validation_loader loop

    if population:
      accuracy = 100.0 * (correct / population)
      record['validation_accuracy'].append(accuracy)

  # end epoch loop
  return record



In [ ]:
n_epoch = 10

result = train_validate(
  model, optimizer, criterion,
  train_loader=train_loader,
  validation_loader=validation_loader,
  n_epoch=n_epoch,
)
result['label'] = 'CNN'



In [ ]:
def plot_result(d_list:Dict[str,List[float]]):
  _, axs = plt.subplots(2, 1)

  for d in d_list:
    try:
      axs[0].plot(d['training_loss'], label=d['label'])
    except TypeError as e:
      print(d)
      raise e
    axs[1].plot(d['validation_accuracy'], label=d['label'])

  axs[0].set_xlabel('iter')
  axs[0].set_ylabel('loss')
  axs[0].legend(loc=0)
  axs[0].grid(True)

  axs[1].set_xlabel('epoch')
  axs[1].set_ylabel('accuracy')
  axs[1].legend(loc=0)
  axs[1].grid(True)



In [ ]:
plot_result([result])



#### GPUs in PyTorch



Runtime / Change runtime type / GPU



In [ ]:
import torch
import torch.cuda

if not torch.cuda.is_available():
  print('CUDA not available in torch')



### 9.5 Torch-Vision Models

